# `Создание и нормализация базы данных`

<img src='data/img/pglogo.png' width=700>

____

### <a id=0>Содержание</a>

- [Нормализация базы данных](#1)
- [Структура базы данных](#2)
- [Подключение к базе данных](#3)
- [Создание таблиц](#4)
- [Загрузка данных](#5)
- [Проблемы при загрузке данных](#4)


___
## <center> <a id=1>Нормализация базы данных</a>

Изначально данные excel-файла находились во `2-й нормальной форме`, т.к.:

- все атрибуты таблицы атомарны

- все строки таблицы уникальны

- имелись ключи для таблиц :

    - transactions - transaction_id

    - customer - customer_id

Таблица не находилась в 3-й нормальной форме т.к.:

- в таблице `transaction` от ключа не зависели столбцы:

    - brand

    - product_line

    - product_class

    - product_size

    - list_price

    - standard_cost

- в таблице `customer` от ключа не зависели столбцы:

    - address

    - postcode

    - state

    - country



В процессе работы, база данных была приведена к `3-й нормальной форме`. Созданы таблицы, в которых каждый столбец зависит только от первичного ключа. 

|Таблица |Ключ |
|:--|:--|
|products |product_id |
| transactions|transaction_id |
| address| postcode|
|customers |customer_id |



___
## <center> <a id=2>Структура базы данных</a>

<img src='data/img/transactions-customers.png'>

[Ссвлка на сайт ](https://dbdiagram.io/d)

<center>

___
## <center> <a id=3>Подключение к базе данных</a>

In [1]:
import psycopg2 as pg
from config import password


conn=pg.connect(
    dbname='transactions_customers',
        user='postgres',
          password=password,
            host='localhost',
                port='5432' )

cursor=conn.cursor()

___
## <center> <a id=4>Создание таблиц</a>

In [2]:
cursor.execute(''' 
CREATE TABLE "transactions" (
  "transaction_id" integer PRIMARY KEY,
  "product_id" integer,
  "customer_id" integer,
  "transaction_date" timestamp,
  "online_order" boolean,
  "order_status" varchar
);

CREATE TABLE "customers" (
  "customer_id" integer PRIMARY KEY,
  "first_name" varchar,
  "last_name" varchar,
  "gender" varchar,
  "DOB" timestamp,
  "job_title" varchar,
  "job_industry_category" varchar,
  "wealth_segment" varchar,
  "deceased_indicator" varchar,
  "owns_car" varchar,
  "property_valuation" integer,
  "postcode" integer
);

CREATE TABLE "address" (
  "postecode" integer PRIMARY KEY,
  "addres" varchar,
  "state" text,
  "country" varchar
);

CREATE TABLE "products" (
  "product_id" integer PRIMARY KEY,
  "brand" varchar,
  "product_line" varchar,
  "product_class" varchar,
  "product_size" varchar,
  "list_price" float,
  "standard_cost" float
);

ALTER TABLE "customers" ADD FOREIGN KEY ("postcode") REFERENCES "address" ("postecode");

ALTER TABLE "transactions" ADD FOREIGN KEY ("customer_id") REFERENCES "customers" ("customer_id");

ALTER TABLE "transactions" ADD FOREIGN KEY ("product_id") REFERENCES "products" ("product_id");

               ''')

conn.commit()


___
## <center> <a id=5>Загрузка данных</a>

### Загрузка в таблицу `address`

<img src='data/img/address.png'>

In [ ]:
f = open(r'data/import data/address_import.csv', 'r')
cursor.copy_from(f, 'address', sep=',')
f.close()

conn.commit()

### Загрузка в таблицу `customers`

<img src='data/img/customers.png'>

In [ ]:
f = open(r'data/import data/customers_import.csv', 'r')
cursor.copy_from(f, 'customers', sep=',')
f.close()

conn.commit()

### Загрузка в таблицу `products`

<img src='data/img/products.png'>

In [54]:
f = open(r'data/import data/product_import.csv', 'r')
cursor.copy_from(f, 'products', sep=',')
f.close()

conn.commit()


### Загрузка в таблицу `transactions`

<img src='data/img/transactions.png'>

In [61]:
f = open(r'data/import data/transaction_import.csv', 'r')
cursor.copy_from(f, 'transactions', sep=',')
f.close()

conn.commit()


___
## <center> <a id=6>Проблемы при загрузке данных</a>
- Для того, чтобы загрузить данные необходимо заполнить `пустые значения` в столбцах :

    - last_name

    - job_title

- Тип данных `boolean` больше подходит для столбцов:
    -  deceased_indicator 

    - owns_car

- Было загружено 10 объектов т.к. :

    - product_id `не уникален` для продуктов, не может быть ключом

